In [1]:
import sys 
import json 
import os 

In [2]:
with open('/Users/raynazheng/Dropbox/Deductive visual question answering with spatial ontology/data/train_sceneGraphs.json') as file:
    train = json.load(file) 
# with open('/Users/raynazheng/Desktop/SceneUnderstanding/2022summer/sceneGraphs/val_sceneGraphs.json') as file:
#     val = json.load(file) 

## Generate Object ID vs Name Dictionary 

In [3]:
obj_dic = {}
for s_id, info in train.items():
    for obj_id, obj_info in info['objects'].items():
        if obj_id not in obj_dic:
            obj_dic[obj_id] = obj_info['name']
        else:
            continue 

with open("object_dictionary.json", "w") as outfile:
    json.dump(obj_dic, outfile)

In [4]:
print(train['4827'])

{'width': 500, 'objects': {'4382361': {'name': 'fork', 'h': 179, 'relations': [{'object': '4382325', 'name': 'near'}, {'object': '4382325', 'name': 'to the left of'}, {'object': '4382325', 'name': 'next to'}], 'w': 30, 'attributes': [], 'y': 162, 'x': 143}, '4382364': {'name': 'table', 'h': 372, 'relations': [], 'w': 499, 'attributes': [], 'y': 3, 'x': 0}, '4382332': {'name': 'spoon', 'h': 28, 'relations': [], 'w': 147, 'attributes': [], 'y': 110, 'x': 187}, '4663512': {'name': 'forks', 'h': 178, 'relations': [{'object': '4663513', 'name': 'on'}, {'object': '4382325', 'name': 'to the left of'}, {'object': '4663513', 'name': 'above'}], 'w': 47, 'attributes': [], 'y': 161, 'x': 123}, '4663513': {'name': 'napkin', 'h': 212, 'relations': [{'object': '4663512', 'name': 'under'}, {'object': '4382325', 'name': 'to the left of'}], 'w': 100, 'attributes': ['cloth', 'white'], 'y': 152, 'x': 86}, '4382325': {'name': 'plate', 'h': 216, 'relations': [{'object': '4663534', 'name': 'to the left of'},

In [5]:
with open("object_dictionary.json", "r") as dic:
    obj_dic = json.load(dic)

## Translate Scene Graph into FOL .in File 

In [7]:
# Assuming 'train' and 'obj_dic' are defined earlier in the code
s_id = '4827'
info = train['4827']

with open(f'scene_{s_id}_1.in', 'w') as f:
    obj_count = 1
    obj_constant_var = {} #dictionary for object identifiers
    connected_objects = set()
    axioms = set()

    for obj_id, obj_info in info['objects'].items():
        # retrive information from scene graph data
        subject = obj_info['name'].replace(" ", "_")
        sub_var = obj_constant_var.setdefault(subject, f'o{obj_count}')
        if sub_var == f'o{obj_count}':
            obj_count += 1
        
        for rela in obj_info['relations']:
            object_id = rela['object']
            if object_id in obj_dic:
                object_name = obj_dic[object_id].replace(" ", "_")
            else:
                continue
            relation = rela['name'].replace(" ", "_")

            obj_var = obj_constant_var.setdefault(object_name, f'o{obj_count}')
            if obj_var == f'o{obj_count}':
                obj_count += 1

            # construct axiom for each relationship
            axiom = f"({subject}({sub_var}) & {relation}({sub_var}, {obj_var}) & {object_name}({obj_var}) & ({sub_var} != {obj_var}))."
            axioms.add(axiom)
            connected_objects.add((subject, object_name))

    #for pairs of objects without explicit spatial relations in the scene graph data
    # the code generates axioms to denote their lack of connection in the scene graph
    for sub, sub_var in obj_constant_var.items():
        for obj, obj_var in obj_constant_var.items():
            if (sub != obj) and ((sub, obj) not in connected_objects):
                axiom = f"({sub}({sub_var}) & -C({sub_var}, {obj_var}) & {obj}({obj_var}) & ({sub_var} != {obj_var}))."
                axioms.add(axiom)
                connected_objects.add((sub, obj))
                connected_objects.add((obj, sub))

    # export into .in file
    for axiom in axioms:
        f.write(axiom + '\n')
        
    f.close()


In [ ]:
import sys 
import json 
import os 

with open('data/train_sceneGraphs.json') as file:
    train = json.load(file) 

obj_dic = {}
for s_id, info in train.items():
    for obj_id, obj_info in info['objects'].items():
        if obj_id not in obj_dic:
            obj_dic[obj_id] = obj_info['name']
        else:
            continue 